In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import geojson
import requests
from bs4 import BeautifulSoup
from yandex_geocoder import Client

import pickle

# yandex developer key
key_ = '16d0b61a-9baa-49c6-8c70-c724ab60214d'


pd.options.display.float_format = '{:.2f}'.format

In [2]:
df = pd.read_csv('../combined_files/cian_flats.csv', sep = ';')
df = df[df.columns[1:]]
df.head()

,id,address,price,floor,square,age
0,212740701,"Москва, ВАО, р-н Соколиная гора, ул. Буракова,...",7099990,12,50,1965
1,207375181,"Москва, ЮВАО, р-н Лефортово, ш. Энтузиастов, 20",13150000,6,62,-1
2,215559357,"Москва, ЮВАО, р-н Нижегородский, Подъемная ул., 6",8100000,3,33,-1
3,215611880,"Москва, ЮВАО, р-н Лефортово, ш. Энтузиастов, 26",11498000,5,62,-1
4,201652911,"Москва, ЮВАО, р-н Лефортово, Красноказарменная...",12467700,2,63,1


In [3]:
try:
    with open('../combined_files/cian_coords.pickle.pkl', 'rb') as handle:
        dict_parsed_flats = pickle.load(handle)
    count_requests = len(dict_parsed_flats) + 2
except:
    dict_parsed_flats = {}
    count_requests = 0

In [4]:
def get_address_coords(address):
    
    """ geocoder """
    if '⋅' in address:
        address = address.split('⋅')[0]

    r = requests.get(f'https://geocode-maps.yandex.ru/1.x/?apikey={key_}&geocode={address}')    
    soup = BeautifulSoup(r.content, 'xml')
    items = soup.find_all('Point')
    l = items[0].text
    return l

In [6]:
for idx, row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
    try:
        if dict_parsed_flats[row['id']] == ['0','0']:
            dict_parsed_flats[row['id']] = get_address_coords(row['address'])  
        if row['id'] not in dict_parsed_flats.keys():
            dict_parsed_flats[row['id']] = get_address_coords(row['address']) 
    except Exception as e:
        print(str(e))
        print(row['address'])
        dict_parsed_flats[row['id']] = ['0','0']
    
    count_requests+=1
    if count_requests%200 == 0:
        f = open("../combined_files/cian_coords.pickle.pkl","wb")
        pickle.dump(dict_parsed_flats,f)
        f.close()  
        
  